In [1]:
import warnings
warnings.simplefilter(action='ignore')
from focal_loss import SparseCategoricalFocalLoss
import pandas as pd
import tensorflow as tf
import glob
import json
import numpy as np
import random
import os
import tensorflow_io as tfio

In [2]:
def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_it_all()

In [3]:
class CFG:
    left_ROWS_per_frame = 21
    sequence_length = 20
    batch_size = 32

labels  = json.load(open('sign_to_prediction_index_map.json','r'))
complete_df = pd.read_csv('train.csv')
from sklearn.model_selection import train_test_split
y = complete_df['sign']
train_df, test_df = train_test_split(complete_df, test_size=0.2,stratify=y)

In [4]:
def build_loader(with_labels=True):
    def load_video(video_path):
        #print('herer')
        video_df = tfio.IODataset.from_parquet(video_path)
        #video_df = pd.read_parquet(video_path, engine='pyarrow')
        #video_df.fillna(0,inplace=True)
        left_df = video_df[video_df.type=='left_hand']
        left_values = left_df[['x','y','z']].values
        left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        right_df = video_df[video_df.type=='right_hand']
        right_values = right_df[['x','y','z']].values
        right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        return [left_hand_array, right_hand_array]
    
    def load_video_with_labels(path, label):
        return load_video(path), labels[label]
    
    return load_video_with_labels if with_labels else load_video

In [11]:
class CustomData(tf.keras.utils.Sequence):
    def __init__(self,df,num_frames=20,batch_size=8,shuffle=True,\
                 labels_path='sign_to_prediction_index_map.json'):
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_frames = num_frames
        self.labels  = json.load(open('sign_to_prediction_index_map.json','r'))
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self,index):
        batches = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        left_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,2))
        right_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,2))
        labels = []
        for i,row_val in enumerate(batches):
            row = self.df.iloc[row_val]
            left_hand,right_hand = self.load_video(row['path'])
            left_hand_input[i,:] = left_hand
            right_hand_input[i,:] = right_hand
            labels.append(self.labels[row['sign']])
        return [left_hand_input,right_hand_input],np.asarray(labels)
            
    def load_video(self,video_path):
        video_df = pd.read_parquet(video_path, engine='pyarrow')
        left_df = video_df[video_df.type=='left_hand']
        left_values = left_df[['x','y']].values
        left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,2)
        if len(left_values)!=0:
            left_values = left_values - np.nanmean(left_values,axis=0)
            left_values = left_values / np.nanstd(left_values,axis=0)
            left_values[np.isnan(left_values)] = 0
            left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        else:
            left_hand_array =  tf.zeros(shape=(CFG.sequence_length, CFG.left_ROWS_per_frame,2),dtype=tf.float32)
        right_df = video_df[video_df.type=='right_hand']
        right_values = right_df[['x','y']].values
        right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,2)
        if len(right_values)!=0:
            right_values = right_values - np.nanmean(right_values,axis=0)
            right_values = right_values / np.nanstd(right_values,axis=0)
            right_values[np.isnan(right_values)] = 0
            right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        else:
            right_hand_array =  tf.zeros(shape=(CFG.sequence_length, CFG.left_ROWS_per_frame,2),dtype=tf.float32)
        return left_hand_array, right_hand_array
    
    def __len__(self):
        return len(self.df)//self.batch_size

In [12]:
train_datagen = CustomData(train_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)
test_datagen = CustomData(test_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)

In [13]:
def conv1d_lstm_block(inputs, filters):
    vector = tf.keras.layers.ConvLSTM1D(filters=32, kernel_size=3,return_sequences=True,padding='same',\
                                        kernel_regularizer=tf.keras.regularizers.L2(l2=0.01),\
                                        recurrent_regularizer=tf.keras.regularizers.L2(l2=0.01))(inputs)
    #vector = tf.keras.layers.Dropout(0.2)(vector)
    vector = tf.keras.layers.BatchNormalization(axis=-1)(vector)
    vector = tf.keras.layers.ConvLSTM1D(filters=64, kernel_size=3,return_sequences=True,padding='same',\
                                        kernel_regularizer=tf.keras.regularizers.L2(l2=0.01),\
                                        recurrent_regularizer=tf.keras.regularizers.L2(l2=0.01))(vector)
    #vector = tf.keras.layers.Dropout(0.2)(vector)
    vector = tf.keras.layers.BatchNormalization(axis=-1)(vector)
    vector = tf.keras.layers.ConvLSTM1D(filters=64, kernel_size=3,padding='same',\
                                       kernel_regularizer=tf.keras.regularizers.L2(l2=0.01),\
                                        recurrent_regularizer=tf.keras.regularizers.L2(l2=0.01))(vector)
    #vector = tf.keras.layers.Dropout(0.2)(vector)
    return vector

def get_model():
    input1 = tf.keras.Input((CFG.sequence_length, CFG.left_ROWS_per_frame, 2), dtype=tf.float32)
    input2 = tf.keras.Input((CFG.sequence_length, CFG.left_ROWS_per_frame, 2), dtype=tf.float32)
    left_hand_vector = conv1d_lstm_block(input1, [64])
    right_hand_vector = conv1d_lstm_block(input2, [64])
    vector = tf.keras.layers.Concatenate(axis=1)([left_hand_vector, right_hand_vector])
    vector = tf.keras.layers.Flatten()(vector)
    vector = tf.keras.layers.Dense(512, activation="relu")(vector)
    vector = tf.keras.layers.Dropout(0.3)(vector)
    output = tf.keras.layers.Dense(250, activation="softmax")(vector)
    model = tf.keras.Model(inputs=[input1,input2], outputs=output)
    model.compile(tf.keras.optimizers.Adam(0.000333),loss=SparseCategoricalFocalLoss(gamma=2), metrics="accuracy")
    return model

In [14]:
model = get_model()

In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 20, 21, 2)]  0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 20, 21, 2)]  0           []                               
                                                                                                  
 conv_lstm1d_6 (ConvLSTM1D)     (None, 20, 21, 32)   13184       ['input_3[0][0]']                
                                                                                                  
 conv_lstm1d_9 (ConvLSTM1D)     (None, 20, 21, 32)   13184       ['input_4[0][0]']                
                                                                                            

In [16]:
file_name = "models/032323_02_12.h5"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        file_name, 
        save_best_only=True, 
        monitor="val_accuracy",
        mode="max",
        verbose = 1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,mode='max',verbose=1,
                              patience=3, min_lr=0.000001)
]
model.fit(train_datagen,validation_data=test_datagen,\
          epochs=30, callbacks=callbacks)

Epoch 1/30
2361/2361 [==============================] - ETA: 0s - loss: 6.2711 - accuracy: 0.0697
Epoch 1: val_accuracy improved from -inf to 0.13332, saving model to models/032323_02_12.h5
2361/2361 [==============================] - 914s 383ms/step - loss: 6.2711 - accuracy: 0.0697 - val_loss: 4.6569 - val_accuracy: 0.1333 - lr: 3.3300e-04
Epoch 2/30
2361/2361 [==============================] - ETA: 0s - loss: 4.3820 - accuracy: 0.1631
Epoch 2: val_accuracy improved from 0.13332 to 0.20127, saving model to models/032323_02_12.h5
2361/2361 [==============================] - 897s 380ms/step - loss: 4.3820 - accuracy: 0.1631 - val_loss: 4.1627 - val_accuracy: 0.2013 - lr: 3.3300e-04
Epoch 3/30
2361/2361 [==============================] - ETA: 0s - loss: 3.9058 - accuracy: 0.2281
Epoch 3: val_accuracy improved from 0.20127 to 0.24650, saving model to models/032323_02_12.h5
2361/2361 [==============================] - 900s 381ms/step - loss: 3.9058 - accuracy: 0.2281 - val_loss: 3.8453 - 

2361/2361 [==============================] - ETA: 0s - loss: 1.1779 - accuracy: 0.7437
Epoch 25: val_accuracy improved from 0.38814 to 0.38840, saving model to models/032323_02_12.h5
2361/2361 [==============================] - 898s 380ms/step - loss: 1.1779 - accuracy: 0.7437 - val_loss: 3.1147 - val_accuracy: 0.3884 - lr: 3.3300e-06
Epoch 26/30
2361/2361 [==============================] - ETA: 0s - loss: 1.1777 - accuracy: 0.7417
Epoch 26: val_accuracy did not improve from 0.38840
2361/2361 [==============================] - 897s 380ms/step - loss: 1.1777 - accuracy: 0.7417 - val_loss: 3.1130 - val_accuracy: 0.3875 - lr: 3.3300e-06
Epoch 27/30
2361/2361 [==============================] - ETA: 0s - loss: 1.1698 - accuracy: 0.7461
Epoch 27: val_accuracy did not improve from 0.38840
2361/2361 [==============================] - 899s 381ms/step - loss: 1.1698 - accuracy: 0.7461 - val_loss: 3.1174 - val_accuracy: 0.3881 - lr: 3.3300e-06
Epoch 28/30
2361/2361 [==============================

In [ ]:
model.save('models/030523_00_09.h5')